In [ ]:
# # Pin NumPy to < 2 and reinstall compatible wheels
# %pip install -q "numpy<2"

# # Force-reinstall all geo deps to match the NumPy ABI
# %pip install -q --force-reinstall \
#   "scipy==1.10.*" \
#   "networkx==2.8.8" \
#   "shapely==2.0.*" \
#   "pyproj==3.6.*" \
#   "rtree==1.2.*" \
#   "fiona==1.9.*" \
#   "geopandas==0.14.*" \
#   "osmnx==1.9.4" \
#   folium

# # (Optional) GEE API if you use the NDVI script
# %pip install -q earthengine-api


In [2]:
print("hello from the top of the script")

hello from the top of the script


GEEE toke: 4/1AVGzR1AS0MVg6BzSSkfBv0SS0xIVdGA7KB0veQckyspF7QvKqr5zTmdmOp8

In [14]:
# narayanganj_green_access_ndvi_osm.py
# Combines Google Earth Engine (Sentinel-2 NDVI) + OSM green areas to compute
# 5/10-minute walking access and propose micro-park candidates in Narayanganj.
#
# Deps:
#   pip install earthengine-api folium osmnx geopandas shapely networkx
#   # If you hit NumPy 2.x ABI issues with GeoPandas/Shapely wheels:
#   # pip install "numpy<2" && pip install --force-reinstall geopandas shapely pyproj fiona rtree
#
# First run will prompt a browser for Earth Engine auth (ee.Authenticate()).

import os
import math
import warnings

# ---- Load heavy libs with a friendly error if NumPy ABI is mismatched ----
try:
    import ee
    import folium
    import networkx as nx
    import osmnx as ox
    import geopandas as gpd
    from shapely.geometry import Point, LineString
    from shapely.ops import unary_union
except Exception as e:
    raise SystemExit(
        f"\nImport error: {e}\n\n"
        "This often happens when GeoPandas/Shapely wheels were built for NumPy 1.x but you're on NumPy 2.x.\n"
        "Quick fix (in a clean venv):\n"
        "  pip install 'numpy<2'\n"
        "  pip install --force-reinstall geopandas shapely pyproj fiona rtree\n"
        "Or use a fresh 'conda create -n aoi python=3.11' env and install the deps.\n"
    )

warnings.filterwarnings("ignore", category=UserWarning)

# ----------------------------
# OSMNX COMPATIBILITY HELPERS
# ----------------------------
try:
    # osmnx >= 2.x
    from osmnx.features import features_from_polygon as osm_features_from_polygon
except Exception:
    try:
        # osmnx <= 1.x
        from osmnx import geometries_from_polygon as osm_features_from_polygon
    except Exception:
        raise SystemExit(
            "Your osmnx version is missing polygon geometries. Please: pip install --upgrade osmnx."
        )

# ----------------------------
# SETTINGS
# ----------------------------
PLACE = "Narayanganj, Dhaka Division, Bangladesh"

# NDVI thresholds
NDVI_GREEN_MIN = 0.35   # pixels with NDVI >= 0.35 are considered 'green' (tweakable)

# Walking thresholds (seconds)
T5 = 5 * 60       # 5 minutes
T10 = 10 * 60     # 10 minutes
WALK_MPS = 1.3    # ~4.7 km/h

# OSM tags to include as green destinations
GREEN_TAGS = {
    "leisure": ["park", "garden"],
    "landuse": ["recreation_ground", "grass"],
    "natural": ["wood"],
}

# Buffer for visualizing reachable edges as polygons (purely cosmetic on the map)
EDGE_BUFFER_M = 25

# Number of micro-park candidates to propose (midpoints of longest uncovered edges)
TOP_N_CANDIDATES = 20

# GEE composite tries (recent → broader → cloudier)
DATE_TRIES = [
    ("2025-09-01", "2025-09-25", 20),
    ("2025-08-01", "2025-09-25", 40),
    ("2025-06-01", "2025-09-25", 80),
]

# Output paths
USER = os.getenv("USER") or os.getenv("USERNAME") or "yourusername"
DOWNLOADS = os.path.expanduser(f"~/Downloads")
OUT_HTML = os.path.join(DOWNLOADS, "narayanganj_green_access_ndvi_osm.html")


# ----------------------------
# EARTH ENGINE HELPERS
# ----------------------------
def ee_init():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()   # opens browser once
        ee.Initialize()


def choose_s2_composite(aoi_geom):
    """Pick a recent, low-cloud Sentinel-2 composite (L2A preferred, fallback to L1C)."""
    for (start, end, cloud) in DATE_TRIES:
        s2sr = (ee.ImageCollection("COPERNICUS/S2_SR")
                .filterBounds(aoi_geom)
                .filterDate(start, end)
                .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud)))
        if s2sr.size().getInfo() > 0:
            return s2sr.median(), f"S2_SR {start}..{end} cloud<{cloud}%"
        s2l1c = (ee.ImageCollection("COPERNICUS/S2")
                 .filterBounds(aoi_geom)
                 .filterDate(start, end)
                 .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", cloud)))
        if s2l1c.size().getInfo() > 0:
            return s2l1c.median(), f"S2_L1C {start}..{end} cloud<{cloud}%"
    raise SystemExit("No recent Sentinel-2 scenes found for AOI after fallbacks.")


def gee_green_polygons(aoi_geom, ndvi_min=NDVI_GREEN_MIN, scale=30, max_features=500):
    """
    Build green polygons from NDVI on GEE:
      - Compute NDVI = (B8 - B4)/(B8 + B4)
      - Threshold NDVI >= ndvi_min
      - Vectorize (reduceToVectors), simplify, and return as GeoJSON-like dict
    """
    composite, desc = choose_s2_composite(aoi_geom)
    print("GEE composite picked:", desc)
    ndvi = composite.normalizedDifference(["B8", "B4"]).rename("NDVI")
    green_mask = ndvi.gte(ndvi_min).selfMask()

    # Vectorize (beware of complexity → use bestEffort + limit)
    vectors = green_mask.reduceToVectors(
        geometry=aoi_geom,
        scale=scale,                 # 10-30 m typical; larger → simpler geometry
        geometryType="polygon",
        labelProperty="class",
        bestEffort=True,
        maxPixels=1e13
    )

    vectors = vectors.limit(max_features)  # keep client payload manageable
    fc = vectors.getInfo()  # bring small feature collection client-side

    # Convert to GeoDataFrame
    feats = fc.get("features", [])
    if not feats:
        return gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")

    geoms = []
    for f in feats:
        geom = f.get("geometry")
        if geom:
            geoms.append(geom)

    gdf = gpd.GeoDataFrame.from_features(feats, crs="EPSG:4326")
    # Keep only polygons
    gdf = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
    return gdf


# ----------------------------
# CORE LOGIC
# ----------------------------
def line_midpoint(geom: LineString):
    try:
        return geom.interpolate(0.5, normalized=True)
    except Exception:
        if geom.geom_type == "LineString" and len(geom.coords) >= 2:
            (x1, y1), (x2, y2) = geom.coords[0], geom.coords[-1]
            return Point((x1 + x2) / 2.0, (y1 + y2) / 2.0)
        return geom.centroid


def make_iso_polygon(edges_subset, buffer_m=EDGE_BUFFER_M):
    if edges_subset.empty:
        return None
    buffered = edges_subset.geometry.buffer(buffer_m)
    merged = unary_union(list(buffered.values))
    return gpd.GeoSeries([merged], crs=edges_subset.crs)


def main():
    # Earth Engine init
    ee_init()

    # OSMnx settings
    ox.settings.log_console = True
    ox.settings.use_cache = True
    ox.settings.timeout = 180

    print("Geocoding AOI…")
    aoi = ox.geocode_to_gdf(PLACE)
    if aoi.empty:
        raise SystemExit("Could not geocode the AOI name.")
    aoi_polygon = aoi.geometry.iloc[0]

    print("Downloading pedestrian network…")
    G = ox.graph_from_polygon(aoi_polygon, network_type="walk", simplify=True)

    print("Projecting graph to local metric CRS…")
    Gp = ox.project_graph(G)
    nodes_gdf, edges_gdf = ox.graph_to_gdfs(Gp)
    if "u" not in edges_gdf.columns or "v" not in edges_gdf.columns:
        edges_gdf = edges_gdf.reset_index()
    graph_crs = nodes_gdf.crs

    # ---- Pull OSM green polygons
    print("Downloading OSM green areas (parks and related tags)…")
    green_layers = []
    for k, v in GREEN_TAGS.items():
        try:
            g = osm_features_from_polygon(aoi_polygon, tags={k: v})
            if g is not None and not g.empty:
                green_layers.append(g)
        except Exception:
            pass
    osm_greens = None
    if green_layers:
        base_crs = getattr(green_layers[0], "crs", None) or "EPSG:4326"
        osm_greens = gpd.GeoDataFrame(
            gpd.pd.concat(green_layers, ignore_index=True), crs=base_crs
        )
        osm_greens = osm_greens[osm_greens.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()

    # ---- Pull NDVI-based green polygons from GEE (limited count/complexity)
    print("Vectorizing NDVI green polygons from GEE (this is server-side fast)…")
    # Build a simple rectangle AOI for GEE from the OSM aoi bounds
    minx, miny, maxx, maxy = aoi.to_crs(epsg=4326).total_bounds
    gee_aoi = ee.Geometry.Rectangle([minx, miny, maxx, maxy])

    ndvi_greens = gee_green_polygons(gee_aoi, ndvi_min=NDVI_GREEN_MIN, scale=30, max_features=600)

    # ---- Merge OSM + NDVI polygons
    greens_list = []
    if osm_greens is not None and not osm_greens.empty:
        greens_list.append(osm_greens.to_crs(epsg=4326))
    if ndvi_greens is not None and not ndvi_greens.empty:
        greens_list.append(ndvi_greens.to_crs(epsg=4326))

    if not greens_list:
        raise SystemExit("No green polygons found from OSM or NDVI. Try lowering NDVI_GREEN_MIN or broadening dates.")
    greens_all = gpd.GeoDataFrame(gpd.pd.concat(greens_list, ignore_index=True), crs="EPSG:4326")
    print(f"Green polygons: OSM={0 if osm_greens is None else len(osm_greens)} | NDVI={len(ndvi_greens)} | merged={len(greens_all)}")

    # Project green polygons to graph CRS
    greens_poly_proj = greens_all.to_crs(graph_crs)

    # Destination nodes = nearest graph nodes to green centroids
    print("Computing destination nodes from green centroids…")
    greens_poly_proj["centroid"] = greens_poly_proj.geometry.centroid
    dest_nodes = set()
    for c in greens_poly_proj["centroid"]:
        try:
            nid = ox.distance.nearest_nodes(Gp, X=c.x, Y=c.y)
            dest_nodes.add(nid)
        except Exception:
            pass

    if not dest_nodes:
        raise SystemExit("No destination nodes mapped from green centroids. Check green layers / AOI.")

    # Edge travel times (seconds)
    print("Assigning time costs to edges…")
    for u, v, k, data in Gp.edges(keys=True, data=True):
        length_m = float(data.get("length", 0.0)) or 0.0
        data["time_s"] = length_m / WALK_MPS

    # Multi-source Dijkstra to nearest green (min time per node)
    print("Running multi-source shortest path (Dijkstra)…")
    Gr = Gp.reverse()  # reverse trick for multi-destination
    min_time_s = nx.multi_source_dijkstra_path_length(Gr, sources=list(dest_nodes), weight="time_s")

    def covered_by_threshold(u, v, threshold_s):
        tu = min_time_s.get(u, math.inf)
        tv = min_time_s.get(v, math.inf)
        return (tu <= threshold_s) or (tv <= threshold_s)

    def both_beyond_10(u, v):
        return (min_time_s.get(u, math.inf) > T10) and (min_time_s.get(v, math.inf) > T10)

    print("Classifying edges by coverage (5 and 10 minutes)…")
    edges_gdf["covered_5min"] = edges_gdf.apply(lambda r: covered_by_threshold(r["u"], r["v"], T5), axis=1)
    edges_gdf["covered_10min"] = edges_gdf.apply(lambda r: covered_by_threshold(r["u"], r["v"], T10), axis=1)
    edges_gdf["uncovered_10min"] = edges_gdf.apply(lambda r: both_beyond_10(r["u"], r["v"]), axis=1)
    uncovered = edges_gdf[edges_gdf["uncovered_10min"]].copy()
    print(f"Uncovered road segments beyond 10 minutes: {len(uncovered)}")

    # Isochrone polygons (from covered edges)
    print("Building isochrone polygons…")
    iso5_edges = edges_gdf[edges_gdf["covered_5min"]]
    iso10_edges = edges_gdf[edges_gdf["covered_10min"]]
    iso5_poly = make_iso_polygon(iso5_edges, buffer_m=EDGE_BUFFER_M)
    iso10_poly = make_iso_polygon(iso10_edges, buffer_m=EDGE_BUFFER_M)

    # Candidate micro-park points: midpoints of longest uncovered segments
    print("Selecting candidate micro-park points…")
    uncovered["length_m"] = uncovered.geometry.length
    candidates = uncovered.sort_values("length_m", ascending=False).head(TOP_N_CANDIDATES).copy()
    candidates["midpt"] = candidates.geometry.apply(line_midpoint)

    # -----------------------
    # Folium map (WGS84)
    # -----------------------
    print("Building Folium map…")
    aoi_latlon = aoi.to_crs(epsg=4326)
    center = [aoi_latlon.geometry.iloc[0].centroid.y, aoi_latlon.geometry.iloc[0].centroid.x]

    edges_latlon = edges_gdf.to_crs(epsg=4326)
    uncovered_latlon = uncovered.to_crs(epsg=4326)
    greens_latlon = greens_poly_proj.to_crs(epsg=4326)
    cand_latlon = gpd.GeoDataFrame(geometry=candidates["midpt"], crs=graph_crs).to_crs(epsg=4326)
    iso5_latlon = iso5_poly.to_crs(epsg=4326) if iso5_poly is not None else None
    iso10_latlon = iso10_poly.to_crs(epsg=4326) if iso10_poly is not None else None

    m = folium.Map(location=center, zoom_start=12, control_scale=True, tiles="cartodbpositron")

    # Green polygons (merged OSM + NDVI)
    folium.GeoJson(
        greens_latlon[["geometry"]],
        name=f"Green areas (OSM + NDVI≥{NDVI_GREEN_MIN:.2f})",
        style_function=lambda _: {"color": "#2e7d32", "weight": 1, "fillColor": "#66bb6a", "fillOpacity": 0.35},
    ).add_to(m)

    # 10-min isochrone
    if iso10_latlon is not None:
        folium.GeoJson(
            iso10_latlon.__geo_interface__,
            name="Within 10 min of green",
            style_function=lambda _: {"color": "#ff9800", "weight": 1, "fillColor": "#ffcc80", "fillOpacity": 0.25},
        ).add_to(m)

    # 5-min isochrone
    if iso5_latlon is not None:
        folium.GeoJson(
            iso5_latlon.__geo_interface__,
            name="Within 5 min of green",
            style_function=lambda _: {"color": "#1976d2", "weight": 1, "fillColor": "#90caf9", "fillOpacity": 0.25},
        ).add_to(m)

    # Uncovered segments (>10 min)
    folium.GeoJson(
        uncovered_latlon[["geometry"]],
        name="Road segments beyond 10 min (need green access)",
        style_function=lambda _: {"color": "#e53935", "weight": 2, "opacity": 0.9},
    ).add_to(m)

    # Candidate micro-park markers — DISTINCT COLOR (bright blue)
    for i, row in cand_latlon.iterrows():
        y, x = row.geometry.y, row.geometry.x
        folium.CircleMarker(
            location=(y, x),
            radius=6,
            color="#2962FF",
            fill=True,
            fill_color="#2962FF",
            fill_opacity=0.95,
            popup=f"Candidate site #{i+1}",
        ).add_to(m)

    # Simple legend
    legend_html = f"""
    <div style="position: fixed; bottom: 18px; left: 18px; z-index:9999; background: white;
                padding: 10px 12px; border: 1px solid #ccc; border-radius: 6px; font-size: 13px;">
      <b>Legend</b><br>
      <span style="display:inline-block;width:12px;height:12px;background:#66bb6a;border:1px solid #2e7d32;"></span>
      Green areas (OSM + NDVI≥{NDVI_GREEN_MIN:.2f})<br>
      <span style="display:inline-block;width:12px;height:12px;background:#ffcc80;border:1px solid #ff9800;"></span>
      ≤ 10 min walk<br>
      <span style="display:inline-block;width:12px;height:12px;background:#90caf9;border:1px solid #1976d2;"></span>
      ≤ 5 min walk<br>
      <span style="display:inline-block;width:18px;height:2px;background:#e53935;vertical-align:middle;display:inline-block;"></span>
      Uncovered roads (>10 min)<br>
      <span style="display:inline-block;width:12px;height:12px;background:#2962FF;border:1px solid #2962FF;"></span>
      Candidate micro-park
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))
    folium.LayerControl(collapsed=False).add_to(m)

    # Save map
    out_cwd = "narayanganj_green_access_ndvi_osm.html"
    m.save(out_cwd)
    print(f"✅ Saved map in current folder: {out_cwd}")

    try:
        os.makedirs(DOWNLOADS, exist_ok=True)
        m.save(OUT_HTML)
        print(f"✅ Also saved to: {OUT_HTML}")
    except Exception as e:
        print("Could not save to ~/Downloads:", e)

    print("Done.")


if __name__ == "__main__":
    main()


Geocoding AOI…


f:\nasa sace app\venv\Lib\site-packages\osmnx\_nominatim.py:65: FutureWarning: `settings.timeout` is deprecated and will be removed in the v2.0.0 release: use `settings.requests_timeout` instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  return _nominatim_request(params=params, request_type=request_type)


f:\nasa sace app\venv\Lib\site-packages\osmnx\_overpass.py:359: FutureWarning: `settings.timeout` is deprecated and will be removed in the v2.0.0 release: use `settings.requests_timeout` instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  overpass_settings = _make_overpass_settings()
f:\nasa sace app\venv\Lib\site-packages\osmnx\_overpass.py:369: FutureWarning: `settings.timeout` is deprecated and will be removed in the v2.0.0 release: use `settings.requests_timeout` instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  yield _overpass_request(data={"data": query_str})


Projecting graph to local metric CRS…


f:\nasa sace app\venv\Lib\site-packages\osmnx\_overpass.py:285: FutureWarning: `settings.timeout` is deprecated and will be removed in the v2.0.0 release: use `settings.requests_timeout` instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  overpass_settings = _make_overpass_settings()
f:\nasa sace app\venv\Lib\site-packages\osmnx\_overpass.py:395: FutureWarning: `settings.timeout` is deprecated and will be removed in the v2.0.0 release: use `settings.requests_timeout` instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  yield _overpass_request(data={"data": query_str})
f:\nasa sace app\venv\Lib\site-packages\osmnx\_overpass.py:285: FutureWarning: `settings.timeout` is deprecated and will be removed in the v2.0.0 release: use `settings.requests_timeout` instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  overpass_settings = _make_overpass_settings()
f:\nasa sace app\venv\Lib\s

Vectorizing NDVI green polygons from GEE (this is server-side fast)…
GEE composite picked: S2_SR 2025-06-01..2025-09-25 cloud<80%
Green polygons: OSM=25 | NDVI=600 | merged=625
Computing destination nodes from green centroids…
Assigning time costs to edges…
Running multi-source shortest path (Dijkstra)…
Classifying edges by coverage (5 and 10 minutes)…
Uncovered road segments beyond 10 minutes: 32888
Building isochrone polygons…
Selecting candidate micro-park points…
Building Folium map…
✅ Saved map in current folder: narayanganj_green_access_ndvi_osm.html
✅ Also saved to: C:\Users\Dipankar Mitra/Downloads\narayanganj_green_access_ndvi_osm.html
Done.


## Shapefile

In [ ]:
import geopandas as gpd
import folium

# -----------------------------
# 1) Load upazila polygons
# -----------------------------
shapefile_path = "Area boundary/Nasa-Area-Boundary/BD_Upazila_BBS21.shp"
gdf = gpd.read_file(shapefile_path)

# Ensure WGS84 (lat/lon) for web maps
if gdf.crs is None:
    # If you know the original CRS, set it here instead of assuming WGS84.
    # Example if it's already lon/lat WGS84:
    gdf = gdf.set_crs(epsg=4326)
else:
    if gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)

# -----------------------------
# 2) Build district boundaries
# -----------------------------
# Dissolve upazilas by district name to get district polygons
dist = gdf[['DISTRICT_N', 'geometry']].dissolve(by='DISTRICT_N', as_index=False)
dist = dist.sort_values('DISTRICT_N').reset_index(drop=True)

# -----------------------------
# 3) Print compact preview
# -----------------------------
# geometry type + bounding box for a quick “print the district boundary” view
preview = dist.copy()
preview["geom_type"] = preview.geometry.geom_type

b = preview.geometry.bounds  # This is a DataFrame with columns: minx, miny, maxx, maxy
preview["bounds"] = list(zip(b["minx"], b["miny"], b["maxx"], b["maxy"]))

print(preview[['DISTRICT_N', 'geom_type', 'bounds']].head(10))

# -----------------------------
# 4) Extract Dhaka district
# -----------------------------
target_name = "Dhaka"
nar = dist[dist["DISTRICT_N"].str.casefold() == target_name.casefold()]

if nar.empty:
    # Helpful hint if the name doesn't match exactly
    available = ", ".join(dist["DISTRICT_N"].head(10).tolist()) + ("..." if len(dist) > 10 else "")
    raise ValueError(
        f"District '{target_name}' not found in DISTRICT_N. "
        f"Example names in your data: {available}"
    )

# Use centroid for initial map centering
centroid = nar.geometry.centroid.iloc[0]  # OK for centering even in geographic CRS
center_latlon = [centroid.y, centroid.x]

# -----------------------------
# 5) Build Folium map
# -----------------------------
m = folium.Map(location=center_latlon, zoom_start=9, tiles="cartodbpositron")

# Layer: all districts (thin outline, no fill)
folium.GeoJson(
    dist.__geo_interface__,
    name="All District Boundaries",
    style_function=lambda feat: {"color": "#555555", "weight": 1, "fill": False},
    tooltip=folium.GeoJsonTooltip(fields=["DISTRICT_N"], aliases=["District"])
).add_to(m)

# Layer: Narayanganj (thicker red outline + light fill)
folium.GeoJson(
    nar.__geo_interface__,
    name="Dhaka",
    style_function=lambda feat: {
        "color": "#d73027",
        "weight": 3,
        "fill": True,
        "fillOpacity": 0.25
    },
    tooltip=folium.GeoJsonTooltip(fields=["DISTRICT_N"], aliases=["District"])
).add_to(m)

# # Marker at Narayanganj centroid
# folium.Marker(
#     location=center_latlon,
#     popup=f"{target_name} District",
#     icon=folium.Icon(color="red", icon="info-sign")
# ).add_to(m)

# # # Fit view to Narayanganj bounds (nice tight framing)
# # minx, miny, maxx, maxy = nar.total_bounds
# # m.fit_bounds([[miny, minx], [maxy, maxx]])

# # folium.LayerControl(collapsed=False).add_to(m)

# # # -----------------------------
# # # 6) Save map
# # # -----------------------------
# # out_html = "bangladesh_districts_narayanganj.html"
# # m.save(out_html)
# # print(f"Map saved to: {out_html}")


        DISTRICT_N     geom_type  \
0         Bagerhat       Polygon   
1        Bandarban       Polygon   
2          Barguna       Polygon   
3         Barishal       Polygon   
4            Bhola       Polygon   
5           Bogura       Polygon   
6     Brahmanbaria       Polygon   
7         Chandpur       Polygon   
8  Chapainababganj       Polygon   
9       Chattogram  MultiPolygon   

                                              bounds  
0  (89.53096578800006, 21.729321597000023, 89.967...  
1  (92.06212887200007, 21.183487962000072, 92.680...  
2  (89.87748492500003, 21.844804644000078, 90.373...  
3  (90.01724254800007, 22.453568824000058, 90.701...  
4  (90.52266803000003, 21.838527058000068, 91.023...  
5  (88.96305583600008, 24.535122898000054, 89.752...  
6  (90.72118216400008, 23.646293997000043, 91.328...  
7  (90.52720869200004, 22.983006697000064, 91.028...  
8  (88.00816912400006, 24.416130831000032, 88.512...  
9  (91.31120302500005, 21.855339392000076, 92.218... 

C:\Users\Dipankar Mitra\AppData\Local\Temp\ipykernel_11708\450428360.py:53: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = nar.geometry.centroid.iloc[0]  # OK for centering even in geographic CRS


In [9]:
preview[dist['DISTRICT_N'] == 'Narayanganj'].geometry

42    POLYGON ((90.52495 23.57348, 90.52442 23.57372...
Name: geometry, dtype: geometry

In [15]:
preview[dist['DISTRICT_N'] == 'Narayanganj'].bounds.iloc[0,:].to_list()

[90.42920437600009, 23.536132402000078, 90.74156867800008, 23.895628744000078]

In [ ]:
# narayanganj_aq_hotspots_readable.py
# Same data/logic as before (S5P NO2, S5P CO, MAIAC AOD -> PM2.5 proxy),
# but with a presentation geared for general users.

import os
import math
from datetime import date, timedelta

import ee
import folium
from folium.plugins import MiniMap, Fullscreen, MousePosition, MeasureControl
from shapely.geometry import Point, MultiPoint, mapping

# ------------------ CONFIG ------------------
# AOI_BBOX = [90.32, 23.70, 90.52, 23.86]  # Narayanganj (W,S,E,N)
        #    [minx, miny, maxx, maxy]
DAYS_BACK = 60
END = date.today()
START = END - timedelta(days=DAYS_BACK)

SCALE_M = 1000
MAX_POINTS = 5000

W_NO2 = 0.6
W_PM25 = 0.6
W_CO  = 0.3

# Hotspot selection
Z_THRESHOLD = 1.0
PCTL_THRESHOLD = 85.0

# Clustering (pure python)
EPS_METERS = 1500.0
MIN_SAMPLES = 6

# Severity buckets (z-score of combined AQ index)
SEVERE_Z = 2.0
HIGH_Z   = 1.0
ELEV_Z   = 0.5

COLORS = {
    "severe": "#d32f2f",
    "high":   "#fb8c00",
    "elev":   "#ffd54f",
    "hull":   "#444444",
    "gray":   "#9e9e9e",
}

USER = os.getenv("USER") or os.getenv("USERNAME") or "user"
OUT_HTML = f"/Users/{USER}/Downloads/narayanganj_aq_hotspots_readable.html"
# --------------------------------------------


TARGET_DISTRICT = "Narayanganj"
polygon = preview[preview['DISTRICT_N'] == TARGET_DISTRICT].geometry.iloc[0]
geoJson = mapping(polygon)
AOI_BBOX = preview[preview['DISTRICT_N'] == 'Narayanganj'].bounds.iloc[0,:].to_list()


def ee_init():
    try: ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()


def build_mean_images(aoi, start_iso, end_iso):
    no2 = (ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2")
           .filterBounds(aoi).filterDate(start_iso, end_iso)
           .select("tropospheric_NO2_column_number_density")
           .mean().rename("no2")).clip(aoi)

    co = (ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CO")
          .filterBounds(aoi).filterDate(start_iso, end_iso)
          .select("CO_column_number_density")
          .mean().rename("co")).clip(aoi)

    aod = (ee.ImageCollection("MODIS/061/MCD19A2_GRANULES")
           .filterBounds(aoi).filterDate(start_iso, end_iso)
           .select("Optical_Depth_047")
           .mean().rename("aod")).clip(aoi)

    pm25 = aod.multiply(60.0).rename("pm25")  # simple proxy
    return no2, pm25, co


def sample_grid(aoi, img_stack, scale_m=SCALE_M, max_points=MAX_POINTS):
    fc = img_stack.sample(region=aoi, scale=scale_m, geometries=True)
    feats = fc.limit(max_points).getInfo().get("features", [])
    rows = []
    for f in feats:
        geom = f.get("geometry", {})
        if geom.get("type") != "Point": continue
        lon, lat = geom["coordinates"]
        p = f.get("properties", {})
        no2, pm25, co = p.get("no2"), p.get("pm25"), p.get("co")
        if None in (no2, pm25, co): continue
        rows.append({"lat": float(lat), "lon": float(lon),
                     "no2": float(no2), "pm25": float(pm25), "co": float(co)})
    return rows


def zscores(vals):
    good = [v for v in vals if v is not None and math.isfinite(v)]
    if len(good) < 2: return [0.0 for _ in vals]
    mean = sum(good)/len(good)
    var  = sum((v-mean)**2 for v in good)/len(good)
    std  = math.sqrt(max(var, 1e-12))
    return [0.0 if (v is None or not math.isfinite(v)) else (v-mean)/std for v in vals]


def p_rank(all_vals, v):
    s = sorted(all_vals)
    if not s: return 0.0
    cnt = sum(1 for x in s if x <= v)
    return 100.0 * cnt / len(s)


def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = p2 - p1
    dl = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dl/2)**2
    return 2*R*math.asin(math.sqrt(a))


def cluster_dbscan(points, eps_m=EPS_METERS, min_samples=MIN_SAMPLES):
    n = len(points)
    visited = [False]*n
    clusters = [-1]*n
    nbrs = [[] for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            if haversine_m(points[i]["lat"], points[i]["lon"], points[j]["lat"], points[j]["lon"]) <= eps_m:
                nbrs[i].append(j); nbrs[j].append(i)
    cid = 0
    for i in range(n):
        if visited[i]: continue
        visited[i] = True
        if len(nbrs[i]) + 1 < min_samples:
            clusters[i] = -1; continue
        clusters[i] = cid
        seeds = list(nbrs[i]); k = 0
        while k < len(seeds):
            j = seeds[k]
            if not visited[j]:
                visited[j] = True
                if len(nbrs[j]) + 1 >= min_samples:
                    for q in nbrs[j]:
                        if q not in seeds: seeds.append(q)
            if clusters[j] < 0: clusters[j] = cid
            k += 1
        cid += 1
    return clusters


def severity_from_z(z):
    if z >= SEVERE_Z: return "severe"
    if z >= HIGH_Z:   return "high"
    if z >= ELEV_Z:   return "elev"
    return None


def build_map(aoi_bbox, hotspots, clusters):
    lat_c = (aoi_bbox[1] + aoi_bbox[3]) / 2.0
    lon_c = (aoi_bbox[0] + aoi_bbox[2]) / 2.0
    m = folium.Map(location=[lat_c, lon_c], zoom_start=12,
                   tiles="cartodbpositron", control_scale=True)

    # Cluster hulls (soft polygons)
    by_cluster = {}
    for hp, cid in zip(hotspots, clusters):
        if cid < 0: continue
        by_cluster.setdefault(cid, []).append(hp)

    for cid, pts in by_cluster.items():
        zs = [p["aq_index_z"] for p in pts]
        avgz = sum(zs)/len(zs)
        sev = severity_from_z(avgz) or "elev"
        color = COLORS[sev]
        # convex hull in WGS84 (approximate)
        mp = MultiPoint([Point(p["lon"], p["lat"]) for p in pts])
        try:
            hull = mp.convex_hull
            folium.GeoJson(
                data=hull.__geo_interface__,
                name=f"Cluster {cid} ({sev}, n={len(pts)})",
                style_function=lambda _,
                                   c=color: {"color": c, "weight": 2, "fillColor": c, "fillOpacity": 0.18},
                tooltip=f"Cluster {cid} — {sev.upper()} (n={len(pts)})"
            ).add_to(m)
        except Exception:
            pass

    # Hotspot markers with intuitive size/color and plain-language popup
    for hp, cid in zip(hotspots, clusters):
        sev = severity_from_z(hp["aq_index_z"])
        if sev is None:
            continue  # hide low-evidence points
        color = COLORS[sev]
        # size by severity
        radius = 6 if sev == "elev" else (8 if sev == "high" else 10)
        hint = []
        if hp["no2_z"] >= 1.0: hint.append("High NO₂ → traffic/industry")
        if hp["pm25_z"] >= 1.0: hint.append("High PM₂.₅ (AOD proxy) → dust/combustion")
        if hp["co_z"]  >= 1.0: hint.append("High CO → combustion/transport")
        hint_text = "<br>".join(hint) if hint else "Mixed drivers"

        folium.CircleMarker(
            location=(hp["lat"], hp["lon"]),
            radius=radius,
            color=color, fill=True, fill_color=color, fill_opacity=0.95,
            tooltip=f"{sev.upper()} hotspot",
            popup=(f"<b>{sev.upper()} hotspot</b><br>"
                   f"AQ index z = {hp['aq_index_z']:.2f} (top {int(hp['percentile'])}%)<br>"
                   f"NO₂ z: {hp['no2_z']:.2f} | PM₂.₅ z: {hp['pm25_z']:.2f} | CO z: {hp['co_z']:.2f}<br>"
                   f"{hint_text}")
        ).add_to(m)

    # Map widgets
    MiniMap(toggle_display=True, position="bottomright").add_to(m)
    Fullscreen().add_to(m)
    MousePosition(position="topright",
                  separator=" | ",
                  prefix="Lat/Lon:").add_to(m)
    MeasureControl(position="topright", primary_length_unit='kilometers').add_to(m)

    # Legend
    legend = """
    <div style="position: fixed; bottom: 18px; left: 18px; z-index:9999; background: white;
                padding: 10px 12px; border: 1px solid #ccc; border-radius: 6px; font-size: 13px;">
      <b>Air-Quality Hotspots (60-day avg)</b><br>
      <span style="display:inline-block;width:12px;height:12px;background:#d32f2f;border:1px solid #d32f2f;"></span>
      Severe (≥ 2σ above AOI mean)<br>
      <span style="display:inline-block;width:12px;height:12px;background:#fb8c00;border:1px solid #fb8c00;"></span>
      High (1–2σ)<br>
      <span style="display:inline-block;width:12px;height:12px;background:#ffd54f;border:1px solid #ffd54f;"></span>
      Elevated (0.5–1σ)<br>
      <i>Shaded polygons show cluster areas.</i>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend))
    folium.LayerControl(collapsed=False).add_to(m)
    return m


def main():
    print("Initializing Earth Engine…")
    ee_init()

    aoi = ee.Geometry(geoJson)
    start_iso, end_iso = str(START), str(END)
    print(f"AOI: {AOI_BBOX} | Window: {start_iso} → {end_iso}")

    # Means
    no2_img, pm25_img, co_img = build_mean_images(aoi, start_iso, end_iso)
    stack = no2_img.addBands(pm25_img).addBands(co_img)

    # Sample grid
    rows = sample_grid(aoi, stack, scale_m=SCALE_M, max_points=MAX_POINTS)
    if not rows:
        raise SystemExit("No samples. Try expanding AOI or increasing DAYS_BACK.")

    # Z-scores
    no2_z = zscores([r["no2"] for r in rows])
    pm25_z = zscores([r["pm25"] for r in rows])
    co_z   = zscores([r["co"] for r in rows])
    aq_raw = [W_NO2*n + W_PM25*p + W_CO*c for n, p, c in zip(no2_z, pm25_z, co_z)]
    aq_index_z = zscores(aq_raw)

    # Pick hotspots (≥1σ or ≥85th percentile)
    def prc(vs, v): return p_rank(vs, v)
    pcts = [prc(aq_index_z, v) for v in aq_index_z]
    hotspots = []
    for r, nz, pz, cz, az, pr in zip(rows, no2_z, pm25_z, co_z, aq_index_z, pcts):
        if (az >= Z_THRESHOLD) or (pr >= PCTL_THRESHOLD):
            hotspots.append({
                "lat": r["lat"], "lon": r["lon"],
                "no2_z": nz, "pm25_z": pz, "co_z": cz,
                "aq_index_z": az, "percentile": pr
            })

    # Cluster
    clusters = cluster_dbscan(hotspots, eps_m=EPS_METERS, min_samples=MIN_SAMPLES) if hotspots else []

    # Map
    m = build_map(AOI_BBOX, hotspots, clusters)
    os.makedirs(os.path.dirname(OUT_HTML), exist_ok=True)
    m.save(OUT_HTML)
    print(f"✅ Saved: {OUT_HTML}")
    

if __name__ == "__main__":
    main()


Initializing Earth Engine…
AOI: [90.42920437600009, 23.536132402000078, 90.74156867800008, 23.895628744000078] | Window: 2025-07-28 → 2025-09-26
✅ Saved: /Users/Dipankar Mitra/Downloads/narayanganj_aq_hotspots_readable.html


In [13]:
# narayanganj_uhi_hotspots.py
# Urban heat island (UHI) hotspots from NASA MODIS LST (daytime), 60-day mean.
# Makes an easy-to-read Folium map with hotspot polygons + markers.

import os
import math
from datetime import date, timedelta

import ee
import folium
from folium.plugins import MiniMap, Fullscreen, MousePosition, MeasureControl
from shapely.geometry import Point, MultiPoint

# ------------- CONFIG -------------
AOI_BBOX = [90.32, 23.70, 90.52, 23.86]  # Narayanganj (W,S,E,N)
DAYS_BACK = 60
END = date.today()
START = END - timedelta(days=DAYS_BACK)

SCALE_M = 1000        # sampling grid; 1000 m ≈ MODIS 1km native
MAX_POINTS = 5000

# Hotspot selection
Z_THRESHOLD = 1.0     # ≥1σ above AOI mean
PCTL_THRESHOLD = 85.0 # or top 15%

# Clustering (pure python DBSCAN)
EPS_METERS = 1500.0
MIN_SAMPLES = 6

# Severity buckets by LST z-score
SEVERE_Z = 2.0
HIGH_Z   = 1.5
ELEV_Z   = 1.0

COLORS = {
    "severe": "#b71c1c",  # dark red
    "high":   "#e53935",  # red
    "elev":   "#fb8c00",  # orange
    "hull":   "#444444",
    "cool":   "#4fc3f7",  # optional cool overlay color
}

USER = os.getenv("USER") or os.getenv("USERNAME") or "user"
OUT_HTML = f"/Users/{USER}/Downloads/narayanganj_uhi_hotspots.html"
# ----------------------------------


def ee_init():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()


def build_lst_day_mean(aoi, start_iso, end_iso):
    """
    Use MODIS/061/MOD11A2 (8-day, 1km) Daytime LST.
    Units: Kelvin * 0.02. Convert to °C and mask invalids.
    """
    coll = (ee.ImageCollection("MODIS/061/MOD11A2")
            .filterBounds(aoi)
            .filterDate(start_iso, end_iso)
            .select("LST_Day_1km"))
    # mask zero or negative (fill) values
    coll = coll.map(lambda img: img.updateMask(img.gt(0)))
    lst_mean_k = coll.mean()  # still in scaled Kelvin
    lst_c = lst_mean_k.multiply(0.02).subtract(273.15).rename("lst_c").clip(aoi)
    return lst_c


def sample_grid(aoi, img, scale_m=SCALE_M, max_points=MAX_POINTS):
    fc = img.sample(region=aoi, scale=scale_m, geometries=True)
    feats = fc.limit(max_points).getInfo().get("features", [])
    rows = []
    for f in feats:
        geom = f.get("geometry", {})
        if geom.get("type") != "Point":
            continue
        lon, lat = geom["coordinates"]
        v = f.get("properties", {}).get("lst_c", None)
        if v is None or not math.isfinite(v):
            continue
        rows.append({"lat": float(lat), "lon": float(lon), "lst_c": float(v)})
    return rows


def zscores(vals):
    good = [v for v in vals if v is not None and math.isfinite(v)]
    if len(good) < 2:
        return [0.0 for _ in vals]
    mean = sum(good)/len(good)
    var  = sum((v-mean)**2 for v in good)/len(good)
    std  = math.sqrt(max(var, 1e-12))
    return [0.0 if (v is None or not math.isfinite(v)) else (v-mean)/std for v in vals]


def p_rank(all_vals, v):
    s = sorted(all_vals)
    if not s: return 0.0
    cnt = sum(1 for x in s if x <= v)
    return 100.0 * cnt / len(s)


def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    p1, p2 = math.radians(lat1), math.radians(lat2)
    dphi = p2 - p1
    dl = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dl/2)**2
    return 2*R*math.asin(math.sqrt(a))


def cluster_dbscan(points, eps_m=EPS_METERS, min_samples=MIN_SAMPLES):
    n = len(points)
    visited = [False]*n
    clusters = [-1]*n
    nbrs = [[] for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            if haversine_m(points[i]["lat"], points[i]["lon"], points[j]["lat"], points[j]["lon"]) <= eps_m:
                nbrs[i].append(j); nbrs[j].append(i)
    cid = 0
    for i in range(n):
        if visited[i]: continue
        visited[i] = True
        if len(nbrs[i]) + 1 < min_samples:
            clusters[i] = -1; continue
        clusters[i] = cid
        seeds = list(nbrs[i]); k = 0
        while k < len(seeds):
            j = seeds[k]
            if not visited[j]:
                visited[j] = True
                if len(nbrs[j]) + 1 >= min_samples:
                    for q in nbrs[j]:
                        if q not in seeds: seeds.append(q)
            if clusters[j] < 0: clusters[j] = cid
            k += 1
        cid += 1
    return clusters


def severity_from_z(z):
    if z >= SEVERE_Z: return "severe"
    if z >= HIGH_Z:   return "high"
    if z >= ELEV_Z:   return "elev"
    return None


def build_map(aoi_bbox, hotspots, clusters):
    lat_c = (aoi_bbox[1] + aoi_bbox[3]) / 2.0
    lon_c = (aoi_bbox[0] + aoi_bbox[2]) / 2.0
    m = folium.Map(location=[lat_c, lon_c], zoom_start=12,
                   tiles="cartodbpositron", control_scale=True)

    # Draw cluster convex hulls (polygons) colored by average severity
    by_cluster = {}
    for hp, cid in zip(hotspots, clusters):
        if cid < 0: continue
        by_cluster.setdefault(cid, []).append(hp)

    for cid, pts in by_cluster.items():
        zs = [p["lst_z"] for p in pts]
        avgz = sum(zs)/len(zs)
        sev = severity_from_z(avgz) or "elev"
        color = COLORS[sev]
        mp = MultiPoint([Point(p["lon"], p["lat"]) for p in pts])
        try:
            hull = mp.convex_hull
            folium.GeoJson(
                data=hull.__geo_interface__,
                name=f"Cluster {cid} ({sev}, n={len(pts)})",
                style_function=lambda _, c=color: {"color": c, "weight": 2,
                                                   "fillColor": c, "fillOpacity": 0.18},
                tooltip=f"UHI Cluster {cid} — {sev.upper()} (n={len(pts)})"
            ).add_to(m)
        except Exception:
            pass

    # Hotspot markers
    for hp, cid in zip(hotspots, clusters):
        sev = severity_from_z(hp["lst_z"])
        if sev is None:
            continue
        color = COLORS[sev]
        radius = 6 if sev == "elev" else (8 if sev == "high" else 10)
        folium.CircleMarker(
            location=(hp["lat"], hp["lon"]),
            radius=radius,
            color=color, fill=True, fill_color=color, fill_opacity=0.95,
            tooltip=f"{sev.upper()} UHI hotspot",
            popup=(f"<b>{sev.upper()} UHI hotspot</b><br>"
                   f"LST: {hp['lst_c']:.1f} °C<br>"
                   f"LST z-score: {hp['lst_z']:.2f} (top {int(hp['percentile'])}%)")
        ).add_to(m)

    # Widgets + legend
    MiniMap(toggle_display=True, position="bottomright").add_to(m)
    Fullscreen().add_to(m)
    MousePosition(position="topright", separator=" | ", prefix="Lat/Lon:").add_to(m)
    MeasureControl(position="topright", primary_length_unit='kilometers').add_to(m)

    legend = f"""
    <div style="position: fixed; bottom: 18px; left: 18px; z-index:9999; background: white;
                padding: 10px 12px; border: 1px solid #ccc; border-radius: 6px; font-size: 13px;">
      <b>Urban Heat Island Hotspots (Daytime LST, {DAYS_BACK}-day mean)</b><br>
      <span style="display:inline-block;width:12px;height:12px;background:{COLORS['severe']};border:1px solid {COLORS['severe']};"></span>
      Severe (≥{SEVERE_Z}σ above AOI mean)<br>
      <span style="display:inline-block;width:12px;height:12px;background:{COLORS['high']};border:1px solid {COLORS['high']};"></span>
      High ({HIGH_Z}–{SEVERE_Z}σ)<br>
      <span style="display:inline-block;width:12px;height:12px;background:{COLORS['elev']};border:1px solid {COLORS['elev']};"></span>
      Elevated ({ELEV_Z}–{HIGH_Z}σ)<br>
      <i>Polygons show hotspot cluster areas.</i>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend))
    folium.LayerControl(collapsed=False).add_to(m)
    return m


def main():
    print("Initializing Earth Engine…")
    ee_init()

    aoi = ee.Geometry.Rectangle(AOI_BBOX)
    start_iso, end_iso = str(START), str(END)
    print(f"AOI: {AOI_BBOX} | Window: {start_iso} → {end_iso}")

    # Build 60-day mean daytime LST (°C)
    lst_img = build_lst_day_mean(aoi, start_iso, end_iso)

    # Sample grid
    print(f"Sampling ~{SCALE_M} m grid (<= {MAX_POINTS} points)…")
    rows = sample_grid(aoi, lst_img, scale_m=SCALE_M, max_points=MAX_POINTS)
    if not rows:
        raise SystemExit("No samples. Try increasing AOI, DAYS_BACK, or MAX_POINTS.")

    # Z-scores and selection
    lst_vals = [r["lst_c"] for r in rows]
    lst_z = zscores(lst_vals)
    pcts  = [p_rank(lst_z, v) for v in lst_z]

    hotspots = []
    for r, z, pr in zip(rows, lst_z, pcts):
        if (z >= Z_THRESHOLD) or (pr >= PCTL_THRESHOLD):
            hotspots.append({
                "lat": r["lat"], "lon": r["lon"],
                "lst_c": r["lst_c"], "lst_z": z, "percentile": pr
            })

    print(f"Hotspot points selected: {len(hotspots)}")

    # Cluster
    clusters = cluster_dbscan(hotspots, eps_m=EPS_METERS, min_samples=MIN_SAMPLES) if hotspots else []

    # Map
    m = build_map(AOI_BBOX, hotspots, clusters)
    os.makedirs(os.path.dirname(OUT_HTML), exist_ok=True)
    m.save(OUT_HTML)
    print(f"✅ Saved UHI map to: {OUT_HTML}\nOpen this file in your browser to explore hotspots.")

if __name__ == "__main__":
    main()


Initializing Earth Engine…
AOI: [90.32, 23.7, 90.52, 23.86] | Window: 2025-07-28 → 2025-09-26
Sampling ~1000 m grid (<= 5000 points)…
Hotspot points selected: 79
✅ Saved UHI map to: /Users/Dipankar Mitra/Downloads/narayanganj_uhi_hotspots.html
Open this file in your browser to explore hotspots.
